# Imports

In [2]:
import sys
import pathlib
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

from sklearn.base import BaseEstimator
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble._forest import ForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

if pathlib.Path().parent.resolve().absolute().as_posix() not in sys.path:
    sys.path.append(pathlib.Path().parent.resolve().absolute().as_posix())

from pilot import Pilot

%load_ext autoreload
%autoreload 2

# Load data

In [2]:
path = "./Bias_correction_ucl.csv"
temp = pd.read_csv(path)
# feature transformation
temp = temp.dropna()
temp["Date"] = pd.to_datetime(temp["Date"])
temp["Day"] = temp["Date"].dt.weekday
temp["Month"] = temp["Date"].dt.month
temp["Year"] = temp["Date"].dt.year
temp.drop("Date", axis=1, inplace=True)
yr = {2013: 1, 2014: 2, 2015: 3, 2016: 4, 2017: 5}
temp["Year"] = temp["Year"].map(yr)
temp.station = temp.station.astype(int)

# define the predictors and the response
X, y = (
            temp.drop(["Next_Tmax", "Next_Tmin"], axis=1).values,
            temp["Next_Tmax"].values,
        )

In [3]:
concrete = pd.read_excel("https://archive.ics.uci.edu/ml/machine-learning-databases/concrete/compressive/Concrete_Data.xls")

In [4]:
concrete_X_train, concrete_X_test, concrete_y_train, concrete_y_test = train_test_split(concrete.iloc[:, :-1], concrete.iloc[:, -1], test_size=0.2)

# Sklearn Random Forest

In [5]:
class PilotRF(ForestRegressor):
    def __init__(self, n_estimators=100, max_depth=12, split_criterion='BIC', min_sample_split=10, min_sample_leaf=5, step_size=1, bootstrap=True, random_state=42, n_jobs=-1):
        super().__init__(
            estimator=Pilot.PILOT(),
            n_estimators=n_estimators,
            estimator_params=('max_depth', 'split_criterion', 'min_sample_split', 'min_sample_leaf', 'step_size'),
            bootstrap=bootstrap,
            n_jobs=n_jobs,
            random_state=random_state
        )
        self.max_depth = max_depth
        self.split_criterion = split_criterion
        self.min_sample_split = min_sample_split
        self.min_sample_leaf = min_sample_leaf
        self.step_size = step_size
        self.criterion = None

In [6]:
model = PilotRF(n_estimators=20, max_depth=12, split_criterion='BIC', min_sample_split=10, min_sample_leaf=5, step_size=1, random_state=42)
model.fit(concrete_X_train, concrete_y_train)
y_pred = model.predict(concrete_X_test)
mean_squared_error(concrete_y_test, y_pred)

40.03437477496381

In [146]:
rf = RandomForestRegressor()
rf.fit(concrete_X_train, concrete_y_train)
y_pred = rf.predict(concrete_X_test)
mean_squared_error(concrete_y_test, y_pred)

28.515534017327113

In [148]:
rf = DecisionTreeRegressor()
rf.fit(concrete_X_train, concrete_y_train)
y_pred = rf.predict(concrete_X_test)
mean_squared_error(concrete_y_test, y_pred)

56.07826990080933

# Custom Random Forest

In [7]:
class RandomForestPilot(BaseEstimator):
    def __init__(
        self,
        n_estimators: int = 10,
        max_depth: int = 12,
        split_criterion: str = 'BIC',
        min_sample_split: int = 10,
        min_sample_leaf: int = 5,
        step_size: int = 1,
        random_state: int = 42
    ):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.split_criterion = split_criterion
        self.min_sample_split = min_sample_split
        self.min_sample_leaf = min_sample_leaf
        self.step_size = step_size
        self.random_state = random_state
        self.estimators = [
            Pilot.PILOT(max_depth, split_criterion, min_sample_split, min_sample_leaf, step_size)
            for i in range(n_estimators)
        ]
    def fit(self, X, y, categorical_idx = np.array([]), n_features: float | str = 'sqrt'):
        X = np.array(X)
        y = np.array(y).reshape(-1, 1)
        n_features = int(np.sqrt(X.shape[1])) if n_features == 'sqrt' else n_features
        for estimator in self.estimators:
            bootstrap_idx = np.random.choice(np.arange(len(X)), size=len(X), replace=True)
            estimator.fit(
                X[bootstrap_idx, :], y[bootstrap_idx], categorical=categorical_idx, max_features_considered=n_features
            )
    
    def predict(self, X) -> np.ndarray:
        X = np.array(X)
        return np.concatenate(
            [e.predict(X).reshape(-1, 1) for e in self.estimators], axis=1
        ).mean(axis=1)
            



In [19]:
pilot_rf = RandomForestPilot(n_estimators=100)
pilot_rf.fit(concrete_X_train, concrete_y_train, categorical_idx=np.array([-1]), n_features=concrete_X_train.shape[1])
y_pred = pilot_rf.predict(concrete_X_test)
mean_squared_error(concrete_y_test, y_pred)

26.599981289829888

In [11]:
print([mean_squared_error(concrete_y_test, e.predict(concrete_X_test)) for e in pilot_rf.estimators])

[45.43339027964507, 39.3376898453851, 40.84999169297349, 37.302813515101185, 46.401611036001064, 46.78663423666957, 39.66845361874538, 38.950197125253446, 47.13128688413894, 40.9015358469173, 47.5625084322141, 35.204434615829584, 55.15378219183933, 40.79892016352728, 41.92408271937778, 37.73435105830955, 41.809410409261254, 50.91261870502332, 51.78335451039562, 48.21974653520454, 43.077168338563375, 40.31990027501892, 37.508765388438476, 48.776592664086934, 30.570356161667696, 35.83876352000485, 35.938546167404084, 45.75365979141959, 32.04746253346872, 46.496916271770125, 45.3040029347239, 52.121612912542936, 50.62062642132253, 49.815265234165565, 46.501151109507724, 46.03276132895037, 43.62157027099284, 39.8855441994921, 47.70023082744096, 34.18448081851126, 40.2972668456168, 43.741845735870584, 38.835876284019584, 46.51424164485817, 54.39004463719353, 38.246871981434914, 39.3075716958859, 42.162787012357974, 43.64426137496962, 47.118142198365724, 44.46569644603736, 45.48037959374095,

In [159]:
n_features = X.shape[1]
n_samples = X.shape[0]



X_new = np.c_[np.arange(0, n_samples, dtype=int), X]

# Memorize the indices of the cases sorted along each feature
# Do not sort the first column since they are just indices
sorted_indices = np.array(
    [
        np.argsort(X_new[:, feature_id], axis=0).flatten()
        for feature_id in range(1, n_features + 1)
    ]
)
sorted_X_indices = (X_new[:, 0][sorted_indices]).astype(int)

sorted_X_indices[0]

array([   0, 3332, 3381, ..., 3056, 3355, 7587])

In [161]:
sorted_indices.shape

(25, 7588)

In [162]:
X.shape

(7588, 25)

In [167]:
sorted_X_indices[0].shape

(7588,)

# Test module

In [5]:
from pilot import ensemble
from sklearn.metrics import r2_score

In [4]:
DATAPATH = pathlib.Path().absolute() / 'Data'
df = pd.read_csv(DATAPATH / 'concrete.csv')
X = df.drop(columns='target').values
y = df['target'].values

In [18]:
rf = ensemble.RandomForestPilot(n_estimators=100)
rf.fit(X, y, n_workers=12)
r2_score(y, rf.predict(X))

0.9420735136186057

In [11]:
rf.estimators[0].predict(X)

AttributeError: 'NoneType' object has no attribute 'node'